[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io/)

# AI Program

## Machine Learning - Deep Learning - PyTorch TensorBoard

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 14/05/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0090DeepLearningPyTorchTensorBoard.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import ParameterGrid

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchinfo
from torchmetrics.classification import MulticlassAccuracy
import torchvision

# Miscellaneous
import copy
from enum import auto, Enum, unique
import math
import os
from platform import python_version
import random
import time

# Typing
from typing import Callable, Dict, Generator, List, Optional, Self, Set, Tuple, Union

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import HTML, Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility (Per PyTorch Version on the same device)
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False #<! Makes things slower


In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

D_CLASSES_SVHN  = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}
L_CLASSES_SVHN  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
T_IMG_SIZE_SVHN = (32, 32, 3)

DATA_FOLDER_PATH    = 'Data'
TENSOR_BOARD_BASE   = 'TB'


In [ ]:
# Download Auxiliary Modules for Google Colab
if runInGoogleColab:
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataManipulation.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataVisualization.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DeepLearningPyTorch.py

In [ ]:
# Courses Packages

from DataVisualization import PlotLabelsHistogram, PlotMnistImages
from DeepLearningPyTorch import NNMode
from DeepLearningPyTorch import RunEpoch


In [ ]:
# General Auxiliary Functions

@unique
class ActivationLayerCls(Enum):
    ELU         = auto()
    GELU        = auto() #<! Common in NLP Transformers
    LEAKY_RELU  = auto()
    RELU        = auto()
    SILU        = auto() #<! Common in NLP Transformers


@unique
class PoolLayerCls(Enum):
    AVERAGE     = auto()
    MAX         = auto()


def PlotConfusionMatrix(vY: np.ndarray, vYPred: np.ndarray, hA: plt.Axes, normMethod: str = None, 
                        lLabels: List = L_CLASSES_SVHN, dScore: Optional[Dict] = None, titleStr: str = 'Confusion Matrix', 
                        xLabelRot: Optional[int] = None, valFormat: Optional[str] = None) -> Tuple[plt.Axes, np.ndarray]:

    hA.clear()

    # Calculation of Confusion Matrix
    mConfMat = confusion_matrix(vY, vYPred, normalize = normMethod)
    oConfMat = ConfusionMatrixDisplay(mConfMat, display_labels = lLabels)
    oConfMat = oConfMat.plot(ax = hA, values_format = valFormat)
    hA = oConfMat.ax_
    if dScore is not None:
        titleStr += ':'
        for scoreName, scoreVal in  dScore.items():
            titleStr += f' {scoreName} = {scoreVal:0.2},'
        titleStr = titleStr[:-1]
    hA.set_title(titleStr)
    hA.grid(False)
    if xLabelRot is not None:
        for xLabel in hA.get_xticklabels():
            xLabel.set_rotation(xLabelRot)

    return hA, mConfMat


## PyTorch & TensorBoard

[TensorBoard](https://www.tensorflow.org/tensorboard) is a tool to analyze runs of models.    
The concept is to save data to HD while running and display it using the server.   
This policy prevents loss of information in case of a failure.  
It also adds the ability to "babysit" the model while running.

Using _TensorBoard_ is based on:

 * Defining a `SummaryWriter` object which documents a session.
 * Using the `SummaryWriter`'s method to add data: Scalars, Figures, Images, etc...


This notebook shows more capabilities of [`SummaryWriter`](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard):

 - Working with a Scalar (`add_scalar()`).
 - Working with Scalars (`add_scalars()`).
 - Working with Figures (`add_figure()`).
 - Working with Hyper Parameters (`add_hparams()`).

</br>

* <font color='brown'>(**#**)</font> While [TensorBoard](https://www.tensorflow.org/tensorboard) is common in the DL world, it might used to handle any ML analysis.
* <font color='brown'>(**#**)</font> See [`torch.utils.tensorboard.writer.SummaryWriter`](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard) documentation.
* <font color='brown'>(**#**)</font> Alternatives: [ClearML](https://clear.ml), [Weights & Biases](https://wandb.ai), [ML Flow](https://mlflow.org), [Neptune AI](https://neptune.ai).
* <font color='brown'>(**#**)</font> [Deep Dive Into TensorBoard: Tutorial With Examples](https://neptune.ai/blog/tensorboard-tutorial).

In [ ]:
# Parameters

# Data
numSamplesPerClsTrain   = 4000
numSamplesPerClsVal     = 400

# Model
dropP = 0.5 #<! Dropout Layer

# Training
batchSize   = 256
numWork     = 2 #<! Number of workers
nEpochs     = 10

# Visualization
numImg = 3


## Generate / Load Data

Load the [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers).  
It is composed of 73,257 RGB train and 26,032 test images of size `32x32`.  
The data is **imbalanced**.

* <font color='brown'>(**#**)</font> The dataset is retrieved using [Torch Vision](https://pytorch.org/vision/stable/index.html)'s built in datasets.  


In [ ]:
# Load Data

# PyTorch 
dsTrain = torchvision.datasets.SVHN(root = DATA_FOLDER_PATH, split = 'train',  download = True, transform = torchvision.transforms.ToTensor())
dsTest  = torchvision.datasets.SVHN(root = DATA_FOLDER_PATH, split = 'test', download = True, transform = torchvision.transforms.ToTensor())
lClass  = np.unique(dsTrain.labels)


print(f'The training data set data shape: {dsTrain.data.shape}')
print(f'The test data set data shape: {dsTest.data.shape}')
print(f'The unique values of the labels: {np.unique(lClass)}')

* <font color='brown'>(**#**)</font> The dataset is indexible (Subscriptable). It returns a tuple of the features and the label.
* <font color='brown'>(**#**)</font> While data is arranged as `H x W x C` the transformer, when accessing the data, will convert it into `C x H x W`. 

In [ ]:
# Element of the Data Set

mX, valY = dsTrain[0]

print(f'The features shape: {mX.shape}')
print(f'The label value: {valY}')

### Plot the Data

In [ ]:
# Extract Data

tX = dsTrain.data #<! NumPy Tensor (NDarray)
mX = np.reshape(tX, (tX.shape[0], -1))
vY = dsTrain.labels #<! NumPy Vector


In [ ]:
# Reorder Data
# Data is C x H x W -> H x W x C for displaying
mX = np.reshape(mX, (mX.shape[0], *T_IMG_SIZE_SVHN[::-1]))
mX = np.transpose(mX, (0, 2, 3, 1))
mX = np.reshape(mX, (mX.shape[0], -1))

In [ ]:
# Plot the Data

hF = PlotMnistImages(mX, vY, numImg, tuImgSize = T_IMG_SIZE_SVHN)

* <font color='red'>(**?**)</font> If data is converted into _grayscale_, how would it effect the performance of the classifier? Explain.  
  You may assume the conversion is done using the mean value of the RGB pixel.

In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(vY, lClass = L_CLASSES_SVHN)
plt.show()

* <font color='red'>(**?**)</font> Explain the given distribution of classes. Think about the origin of the data.

## Pre Process Data

This section:

 * Normalizes the data in a predefined manner.
 * Takes a sub set of the data.

In [ ]:
# Sub Set of Data - Indices

numCls = len(L_CLASSES_SVHN)
mIdxTrain   = np.zeros(shape = (numSamplesPerClsTrain, numCls), dtype = np.int_)
mIdxVal     = np.zeros(shape = (numSamplesPerClsVal, numCls), dtype = np.int_)

for valCls in range(numCls):
    mIdxTrain[:, valCls] = np.random.choice(np.flatnonzero(dsTrain.labels == valCls), size = numSamplesPerClsTrain, replace = False)
    mIdxVal[:, valCls] = np.random.choice(np.flatnonzero(dsTest.labels == valCls), size = numSamplesPerClsVal, replace = False)


In [ ]:
# Sub Set of Data - Subset
dsTrain = torch.utils.data.Subset(dsTrain, np.ravel(mIdxTrain))
dsTest = torch.utils.data.Subset(dsTest, np.ravel(mIdxVal))

In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(dsTrain.dataset.labels[dsTrain.indices], lClass = L_CLASSES_SVHN)
plt.show()

In [ ]:
hA = PlotLabelsHistogram(dsTest.dataset.labels[dsTest.indices], lClass = L_CLASSES_SVHN)
plt.show()

In [ ]:
# Update Transformer

oDataTrns = torchvision.transforms.Compose([    #<! Chaining transformations
    torchvision.transforms.ToTensor(),          #<! Convert to Tensor (C x H x W), Normalizes into [0, 1] (https://pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html)
    torchvision.transforms.Normalize(0.5, 0.5),
    ])

# Update the DS transformer
dsTrain.transform = oDataTrns
dsTest.transform  = oDataTrns

* <font color='red'>(**?**)</font> What does the `Normalize(0.5, 0.5)` do? What are the value boundaries of the output?

In [ ]:
# "Normalized" Image

mX, valY = dsTrain[5]

hF, hA = plt.subplots()
hImg = hA.imshow(np.transpose(mX, (1, 2, 0)))
hF.colorbar(hImg)
plt.show()

### Data Loaders

This section defines the data loaded.

* <font color='brown'>(**#**)</font> Adjust the size of the actual data set according to performance of the system.
* <font color='brown'>(**#**)</font> First tuning and runs of a script might be better done with a small data.  
* <font color='brown'>(**#**)</font> Sub sampling the data can be achieved using `torch.utils.data.Subset`.  



In [ ]:
# Data Loader

dlTrain  = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = numWork, persistent_workers = True)
dlTest   = torch.utils.data.DataLoader(dsTest, shuffle = False, batch_size = 2 * batchSize, num_workers = numWork, persistent_workers = True)


* <font color='red'>(**?**)</font> Why is the size of the batch twice as big for the test dataset?

In [ ]:
# Iterate on the Loader
# The first batch.
tX, vY = next(iter(dlTrain)) #<! PyTorch Tensors

print(f'The batch features dimensions: {tX.shape}')
print(f'The batch labels dimensions: {vY.shape}')

In [ ]:
# Looping
for ii, (tX, vY) in zip(range(1), dlTest): #<! https://stackoverflow.com/questions/36106712
    print(f'The batch features dimensions: {tX.shape}')
    print(f'The batch labels dimensions: {vY.shape}')

## Define the Model

The model is defined as a sequential model.

In [ ]:
# Model
# Defining a sequential model.

numFeatures = np.prod(tX.shape[1:])

def BuildModel( activationLyrType: ActivationLayerCls = ActivationLayerCls.RELU, poolLayerType: PoolLayerCls = PoolLayerCls.MAX ) -> nn.Module:

    if activationLyrType is ActivationLayerCls.ELU:
        ActivationLayer = nn.ELU
    elif activationLyrType is ActivationLayerCls.GELU:
        ActivationLayer = nn.GELU
    elif activationLyrType is ActivationLayerCls.LEAKY_RELU:
        ActivationLayer = nn.LeakyReLU
    elif activationLyrType is ActivationLayerCls.RELU:
        ActivationLayer = nn.ReLU
    elif activationLyrType is ActivationLayerCls.SILU:
        ActivationLayer = nn.SiLU
    else:
        raise ValueError(f'The value of `activationLyrType` {activationLyrType} is not supported')
    
    if poolLayerType is PoolLayerCls.AVERAGE:
        PoolLayer = nn.AvgPool2d
    elif poolLayerType is PoolLayerCls.MAX:
        PoolLayer = nn.MaxPool2d
    else:
        raise ValueError(f'The value of `poolLayerType` {activationLyrType} is not supported')
    
    oModel = nn.Sequential(
        nn.Identity(),
        
        nn.Conv2d(3,    16, 3, bias = False), nn.BatchNorm2d(16),                ActivationLayer(),
        nn.Conv2d(16,   32, 3, bias = False), nn.BatchNorm2d(32),  PoolLayer(2), ActivationLayer(),
        nn.Conv2d(32,   64, 3, bias = False), nn.BatchNorm2d(64),  PoolLayer(2), ActivationLayer(),
        nn.Conv2d(64,  128, 3, bias = False), nn.BatchNorm2d(128),               ActivationLayer(),
        nn.Conv2d(128, 256, 3, bias = False), nn.BatchNorm2d(256),               ActivationLayer(),
        
        nn.AdaptiveAvgPool2d(1),
        nn.Flatten(),
        nn.Linear(256, len(lClass)),
    )

    return oModel

oModel = BuildModel()

torchinfo.summary(oModel, tX.shape, col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu')

* <font color='red'>(**?**)</font> Why is `bias = False` used above?
* <font color='brown'>(**#**)</font> Using a multiplication by 8 number of channels accelerate the run time (In most cases).
* <font color='brown'>(**#**)</font> Pay attention to model size and the RAM fo the GPU. Rule of thumb, up to ~40%.

## Train the Model

This section trains the model using different schedulers:

 - Updates the training function to use more features of _TensorBoard_.
 - Trains the model with different _hyper parameters_.

In [ ]:
# Run Device

runDevice = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device

In [ ]:
# Loss and Score Function

hL = nn.CrossEntropyLoss()
hS = MulticlassAccuracy(num_classes = len(lClass))
hL = hL.to(runDevice) #<! Not required!
hS = hS.to(runDevice)

In [ ]:
# Train Model

def TrainModel( oModel: nn.Module, dlTrain: DataLoader, dlVal: DataLoader, oOpt: Optimizer, numEpoch: int, hL: Callable, hS: Callable, oSch: Optional[LRScheduler] = None, oTBWriter: Optional[SummaryWriter] = None) -> Tuple[nn.Module, List, List, List, List]:

    lTrainLoss  = []
    lTrainScore = []
    lValLoss    = []
    lValScore   = []
    lLearnRate  = []

    # Support R2
    bestScore = -1e9 #<! Assuming higher is better

    learnRate = oOpt.param_groups[0]['lr']

    for ii in range(numEpoch):
        startTime           = time.time()
        trainLoss, trainScr = RunEpoch(oModel, dlTrain, hL, hS, oOpt, opMode = NNMode.TRAIN) #<! Train
        valLoss,   valScr   = RunEpoch(oModel, dlVal, hL, hS, oOpt, opMode = NNMode.INFERENCE) #<! Score Validation
        if oSch is not None:
            # Adjusting the scheduler on Epoch level
            learnRate = oSch.get_last_lr()[0]
            oSch.step()
        epochTime           = time.time() - startTime

        # Aggregate Results
        lTrainLoss.append(trainLoss)
        lTrainScore.append(trainScr)
        lValLoss.append(valLoss)
        lValScore.append(valScr)
        lLearnRate.append(learnRate)

        if oTBWriter is not None:
            oTBWriter.add_scalars('Loss (Epoch)', {'Train': trainLoss, 'Validation': valLoss}, ii)
            oTBWriter.add_scalars('Score (Epoch)', {'Train': trainScr, 'Validation': valScr}, ii)
            oTBWriter.add_scalar('Learning Rate (Epoch)', learnRate, ii)

            runDevice = next(oModel.parameters()).device
            oModel.eval()
            lYHat   = []
            lY      = []
            for jj, (mX, vY) in enumerate(dlVal):
                mX = mX.to(runDevice) #<! Lazy

                with torch.inference_mode():
                    mZ = oModel(mX)
                    vYHat = torch.argmax(mZ, dim = 1)
                
                lYHat.extend(vYHat.cpu().numpy())
                lY.extend(vY.numpy())
            
            vYHat   = np.array(lYHat)
            vY      = np.array(lY)

            hF, hA = plt.subplots(figsize = (9, 5)) #<! Complex axes, hence `cla()` won't do it
            hA, _ = PlotConfusionMatrix(vY, vYHat, hA)
            oTBWriter.add_figure('Confusion Matrix (Epoch)', hF, ii, close = True)
        
        # Display (Babysitting)
        print('Epoch '              f'{(ii + 1):4d} / ' f'{numEpoch}:', end = '')
        print(' | Train Loss: '     f'{trainLoss          :6.3f}', end = '')
        print(' | Val Loss: '       f'{valLoss            :6.3f}', end = '')
        print(' | Train Score: '    f'{trainScr           :6.3f}', end = '')
        print(' | Val Score: '      f'{valScr             :6.3f}', end = '')
        print(' | Epoch Time: '     f'{epochTime          :5.2f}', end = '')

        # Save best model ("Early Stopping")
        if valScr > bestScore:
            bestScore = valScr
            print(' | <-- Checkpoint!', end = '')
            try:
                dCheckpoint = {'Model' : oModel.state_dict(), 'Optimizer' : oOpt.state_dict()}
                if oSch is not None:
                    dCheckpoint['Scheduler': oSch.state_dict()]
                torch.save(dCheckpoint, 'BestModel.pt')
            except:
                pass
        print(' |')
    
    # Load best model ("Early Stopping")
    # dCheckpoint = torch.load('BestModel.pt')
    # oModel.load_state_dict(dCheckpoint['Model'])

    return oModel, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate


* <font color='brown'>(**#**)</font> Some schedulers (For instance `OneCycleLR`) do not allow iterations beyond what is defined.
* <font color='brown'>(**#**)</font> Some schedulers are score / loss event driven. See `torch.optim.ReduceLROnPlateau`.

In [ ]:
# Parameters Grid

dParamGrid = ParameterGrid({'activationLyrType': [ActivationLayerCls.ELU, ActivationLayerCls.GELU, ActivationLayerCls.LEAKY_RELU, ActivationLayerCls.RELU, ActivationLayerCls.SILU], 
                            'poolLayerType': [PoolLayerCls.AVERAGE, PoolLayerCls.MAX]})

In [ ]:
# Train Model

learnRate = 4e-3

dModelHist = {}

for ii, dModelParams in enumerate(dParamGrid):
    print(f'Training with the {(ii + 1): 02d} model parameters combination')
    oTBWriter = SummaryWriter(log_dir = os.path.join(TENSOR_BOARD_BASE, f'Model{(ii + 1):03d}'))
    oModel = BuildModel(**dModelParams) #<! Just for graphing
    oTBWriter.add_graph(oModel, tX) #<! Model Graph
    oModel = BuildModel(**dModelParams)
    oModel = oModel.to(runDevice) #<! Transfer model to device
    oOpt = torch.optim.AdamW(oModel.parameters(), lr = learnRate, betas = (0.9, 0.99), weight_decay = 1e-4) #<! Define optimizer
    oScd = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr = learnRate, total_steps = nEpochs)
    _, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate = TrainModel(oModel, dlTrain, dlTest, oOpt, nEpochs, hL, hS, oScd, oTBWriter)
    oTBWriter.add_hparams({'Activation Layer': dModelParams['activationLyrType'].name, 'Pool Layer': dModelParams['poolLayerType'].name}, 
                          {'Loss (Train)': min(lTrainLoss), 'Score (Train)': max(lTrainScore), 'Loss (Validation)': min(lValLoss), 'Score (Validation)': max(lValScore)})
    dModelHist[ii] = lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate
    oTBWriter.close()
    

* <font color='blue'>(**!**)</font> Plot the data (`dModelHist`) using _MatPlotLib_.
* <font color='blue'>(**!**)</font> Add an image of some of mislabeled images. See [PyTorch TensorBoard Tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html).
* <font color='blue'>(**!**)</font> Add a PR Curve for the TB result. See [PyTorch TensorBoard Tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html).